In [20]:
# Import libs
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage

In [21]:
# GCS details
gcs= pa.fs.GcsFileSystem()
bucket_name = 'de-zoomcamp-dominik-1'
project_id = "dtc-de-course-412911"
table_name = "fhv_trips_data"
root_path = f'{bucket_name}/{table_name}'

In [23]:
years =  [2019]
months = range(1,13)
for year in years:
    for month in months: 
        taxi_dtypes = {
                            'dispatching_base_num': str,
                            'PULocationID': pd.Int64Dtype(),
                            'DOLocationID': pd.Int64Dtype(),
                            'SR_Flag': float,
                            'Affiliated_base_number':str
                        }    
        parse_dates = ['pickup_datetime', 'dropOff_datetime']
        url = f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_{year}-{month:02}.csv.gz'
        df = pd.read_csv(url, dtype=taxi_dtypes, sep=',', compression='gzip', parse_dates=parse_dates, low_memory=False)
        
        # Column for partitioning by pickup time
        df['pickup_date'] = df['pickup_datetime'].dt.strftime("%Y%m%d")
        df['pickup_datetime'] = df['pickup_datetime'].dt.strftime("%Y-%m-%d %H:%M:%S")
        df['dropOff_datetime'] = df['dropOff_datetime'].dt.strftime("%Y-%m-%d %H:%M:%S")
        
        # Write do GCS Bucket
        table = pa.Table.from_pandas(df)
        pq.write_to_dataset(
                table,
                root_path = root_path,
                partition_cols = ['pickup_date'],
                filesystem=gcs)
        print(f'{month} - {year} added go GCS!')

1 - 2019 added go GCS!
2 - 2019 added go GCS!
3 - 2019 added go GCS!
4 - 2019 added go GCS!
5 - 2019 added go GCS!
6 - 2019 added go GCS!
7 - 2019 added go GCS!
8 - 2019 added go GCS!


ArrowException: Unknown error: google::cloud::Status(UNKNOWN: Retry policy exhausted UploadChunk: PerformWork() - CURL error [65]=Send failed since rewinding of the data stream failed)